In [3]:
import re
import os
import sys
import exceptions
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
%matplotlib inline

---

In [5]:
# get files in directory
def get_files(src_dir):
    f = []
    for (dirpath, dirnames, filenames) in os.walk(src_dir):
        f.extend(filenames)
        break
    return f

# create graph from pb file
def create_graph(pb_file, sess):
    with sess:        
        with tf.gfile.FastGFile(pb_file, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, name='')
    return sess.graph

---

In [6]:
INPUT_DIR = 'out/crops'
OUTPUT_DIR_BOTTLENECKS = 'out/bottlenecks'

In [7]:
if not os.path.isdir(OUTPUT_DIR_BOTTLENECKS): os.makedirs(OUTPUT_DIR_BOTTLENECKS)

In [8]:
# init tf sess
sess = tf.Session()

# restore inceptionv3 graph
create_graph('/models/inception-v3.pb', sess)

# summary writer for tensorboard
summary_writer = tf.train.SummaryWriter('/tmp/tf-painters-v3', graph=sess.graph)

In [9]:
# find tensors we're interested in
bottneck_t = sess.graph.get_tensor_by_name('pool_3/_reshape:0')
decode_jpeg_t = sess.graph.get_tensor_by_name('DecodeJpeg/contents:0')

In [ ]:
files = [INPUT_DIR + '/' + f for f in get_files(INPUT_DIR)]

In [29]:
for f in tqdm(files):
    fn = re.findall(r'([^/]+)\.jpg', f)[0]
    bn_f = OUTPUT_DIR_BOTTLENECKS + '/' + fn + '.npy'
    
    if not os.path.isfile(bn_f):
        image_data = tf.gfile.FastGFile(f, 'rb').read()
        bn = sess.run(bottneck_t, {decode_jpeg_t: image_data})
        bn = np.squeeze(bn)
        np.save(bn_f, bn)

100%|██████████| 516250/516250 [3:32:45<00:00, 40.44it/s]
